# 🔹 UFC Deployment Notebook

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Import Libraries and Setup Environment

In [1]:
# Import necessary libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 200) 

# Get the current working directory
current_dir = os.getcwd()

# Navigate to the project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Import from /src
sys.path.append(os.path.join(project_root))
from src.metrics import *
from src.model_factory import model_factory
from src.model import UFCModel
from src.data import UFCData
from src.predictor import UFCPredictor
from src.config import *
from src.io_model import *
from src.helpers import *

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Load Data

In [2]:
# Load UFCData
try:
    ufc_data = load_data(name='ufc_data')
    ufc_data_no_odds = load_data(name='ufc_data_no_odds')
    logger.info("✅ UFCData objects loaded successfully.")
except Exception as e:
    logger.error(f"❌ Error loading training data: {e}")

INFO:src.io_model:📦 UFCData object loaded from: /home/mlioi/ufc-predictor/data/processed/ufc_data.pkl
INFO:src.io_model:📦 UFCData object loaded from: /home/mlioi/ufc-predictor/data/processed/ufc_data_no_odds.pkl
INFO:src.helpers:✅ UFCData objects loaded successfully.


In [3]:
ufc_data

📊 UFC Dataset Summary
----------------------------------------
🧪 Total samples      : 6001
🧪 Train/Test split  : 4800 / 1201
🧪 Total features     : 28

🔢 Numerical features : 25
🔠 Categorical features: 3
    - Binary          : 2
    - Multiclass      : 1

🏷 Label distribution (raw):
   - Class 0: 3484 (58.1%)
   - Class 1: 2517 (41.9%)

✅ No missing values detected

📈 Feature summary statistics (train set):
                        mean      std      min       max
BlueTotalTitleBouts    0.260    1.111     0.00    16.000
RedTotalTitleBouts     0.572    1.574     0.00    16.000
LoseStreakDif          0.059    1.012    -6.00     6.000
WinStreakDif          -0.162    1.921   -18.00    10.000
LongestWinStreakDif   -0.772    2.053   -12.00    14.000
KODif                 -0.536    2.181   -21.00    14.000
SubDif                -0.334    1.877   -15.00    10.000
HeightDif             -0.030    6.286   -33.02    30.480
ReachDif              -0.260    8.266   -33.02    30.480
AgeDif            

In [4]:
ufc_data_no_odds

📊 UFC Dataset Summary
----------------------------------------
🧪 Total samples      : 6001
🧪 Train/Test split  : 4800 / 1201
🧪 Total features     : 27

🔢 Numerical features : 24
🔠 Categorical features: 3
    - Binary          : 2
    - Multiclass      : 1

🏷 Label distribution (raw):
   - Class 0: 3484 (58.1%)
   - Class 1: 2517 (41.9%)

✅ No missing values detected

📈 Feature summary statistics (train set):
                      mean     std     min      max
BlueTotalTitleBouts  0.260   1.111    0.00   16.000
RedTotalTitleBouts   0.572   1.574    0.00   16.000
LoseStreakDif        0.059   1.012   -6.00    6.000
WinStreakDif        -0.162   1.921  -18.00   10.000
LongestWinStreakDif -0.772   2.053  -12.00   14.000
KODif               -0.536   2.181  -21.00   14.000
SubDif              -0.334   1.877  -15.00   10.000
HeightDif           -0.030   6.286  -33.02   30.480
ReachDif            -0.260   8.266  -33.02   30.480
AgeDif              -0.611   5.181  -17.00   16.000
SigStrDif       

In [5]:
# Define the path to the CSV file
file_path = os.path.join(project_root, 'data', 'processed', 'ufc_deploy.csv')

# Load the CSV into a DataFrame
try:
    ufc_df = pd.read_csv(file_path)
    logger.info(f"✅ Data successfully loaded: {ufc_df.shape[0]} rows, {ufc_df.shape[1]} columns.")
except Exception as e:
    logger.error(f"❌ Error loading data: {e}")

INFO:src.helpers:✅ Data successfully loaded: 4719 rows, 82 columns.


# Load Models

In [6]:
model_list = load_all_models()

INFO:src.io_model:📦 Model 'Logistic Regression' loaded from: /home/mlioi/ufc-predictor/models/lr_best.pkl
INFO:src.io_model:📦 Model 'Logistic Regression' loaded from: /home/mlioi/ufc-predictor/models/lr_best_no_odds.pkl
INFO:src.io_model:📦 Model 'Random Forest' loaded from: /home/mlioi/ufc-predictor/models/rf_best.pkl
INFO:src.io_model:📦 Model 'Random Forest' loaded from: /home/mlioi/ufc-predictor/models/rf_best_no_odds.pkl
INFO:src.io_model:📦 Model 'Support Vector Machine' loaded from: /home/mlioi/ufc-predictor/models/svm_best.pkl
INFO:src.io_model:📦 Model 'Support Vector Machine' loaded from: /home/mlioi/ufc-predictor/models/svm_best_no_odds.pkl
INFO:src.io_model:📦 Model 'K-Nearest Neighbors' loaded from: /home/mlioi/ufc-predictor/models/knn_best.pkl
INFO:src.io_model:📦 Model 'K-Nearest Neighbors' loaded from: /home/mlioi/ufc-predictor/models/knn_best_no_odds.pkl
INFO:src.io_model:📦 Model 'AdaBoost' loaded from: /home/mlioi/ufc-predictor/models/ab_best.pkl
INFO:src.io_model:📦 Model '

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Create WeightGroup Mapping

In [7]:
ufc_df = ufc_df[ufc_df['WeightClass'] != 'Catch Weight']

In [8]:
ufc_df['WeightClass'].unique()

array(['Welterweight', 'Featherweight', 'Flyweight', 'Light Heavyweight',
       'Bantamweight', 'Lightweight', "Women's Flyweight",
       "Women's Strawweight", 'Heavyweight', 'Middleweight',
       "Women's Bantamweight", "Women's Featherweight"], dtype=object)

In [9]:
# Diccionario de mapeo de WeightClass → WeightGroup
weight_class_map = {
    'Flyweight': 'Light',
    'Bantamweight': 'Light',
    'Featherweight': 'Light',
    'Lightweight': 'Light',
    'Welterweight': 'Medium',
    'Middleweight': 'Medium',
    'Light Heavyweight': 'Heavy',
    'Heavyweight': 'Heavy',
    "Women's Flyweight": 'Women',
    "Women's Strawweight": 'Women',
    "Women's Bantamweight": 'Women',
    "Women's Featherweight": 'Women',
}
# Crear nueva columna con el grupo
ufc_df['WeightClassMap'] = ufc_df['WeightClass'].map(weight_class_map)

In [10]:
ufc_df.columns

Index(['RedFighter', 'BlueFighter', 'RedOdds', 'BlueOdds', 'RedExpectedValue',
       'BlueExpectedValue', 'Date', 'Location', 'Country', 'TitleBout',
       'WeightClass', 'Gender', 'NumberOfRounds', 'BlueCurrentLoseStreak',
       'BlueCurrentWinStreak', 'BlueDraws', 'BlueAvgSigStrLanded',
       'BlueAvgSigStrPct', 'BlueAvgSubAtt', 'BlueAvgTDLanded', 'BlueAvgTDPct',
       'BlueLongestWinStreak', 'BlueLosses', 'BlueTotalRoundsFought',
       'BlueTotalTitleBouts', 'BlueWinsByDecisionMajority',
       'BlueWinsByDecisionSplit', 'BlueWinsByDecisionUnanimous',
       'BlueWinsByKO', 'BlueWinsBySubmission', 'BlueWinsByTKODoctorStoppage',
       'BlueWins', 'BlueStance', 'BlueHeightCms', 'BlueReachCms',
       'BlueWeightLbs', 'RedCurrentLoseStreak', 'RedCurrentWinStreak',
       'RedDraws', 'RedAvgSigStrLanded', 'RedAvgSigStrPct', 'RedAvgSubAtt',
       'RedAvgTDLanded', 'RedAvgTDPct', 'RedLongestWinStreak', 'RedLosses',
       'RedTotalRoundsFought', 'RedTotalTitleBouts',
       'RedWi

In [11]:
ufc_df

,RedFighter,BlueFighter,RedOdds,BlueOdds,RedExpectedValue,BlueExpectedValue,Date,Location,Country,TitleBout,...,SigStrDif,AvgSubAttDif,AvgTDDif,BetterRank,Finish,FinishRound,FinishRoundTime,TotalFightTimeSecs,label,WeightClassMap
0,Colby Covington,Joaquin Buckley,205.0,-250.0,205.0000,40.0000,2024-12-14,"Tampa, Florida, USA",USA,False,...,0.2500,-0.2000,-1.8300,Red,KO/TKO,3.0,4:42,882.0,1,Medium
1,Cub Swanson,Billy Quarantillo,124.0,-148.0,124.0000,67.5676,2024-12-14,"Tampa, Florida, USA",USA,False,...,2.6900,0.7000,0.2000,neither,KO/TKO,3.0,1:36,696.0,0,Light
2,Manel Kape,Bruno Silva,-395.0,310.0,25.3165,310.0000,2024-12-14,"Tampa, Florida, USA",USA,False,...,-1.1200,-0.2000,1.7200,Red,KO/TKO,3.0,1:57,717.0,0,Light
3,Vitor Petrino,Dustin Jacoby,-340.0,270.0,29.4118,270.0000,2024-12-14,"Tampa, Florida, USA",USA,False,...,2.6800,-0.8000,-3.6200,neither,KO/TKO,3.0,3:44,824.0,1,Heavy
4,Adrian Yanez,Daniel Marcos,185.0,-225.0,185.0000,44.4444,2024-12-14,"Tampa, Florida, USA",USA,False,...,-0.5700,0.0000,0.2500,neither,S-DEC,3.0,5:00,900.0,1,Light
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4714,Alessio Sakara,James Irvin,-120.0,100.0,83.3333,100.0000,2010-03-21,"Broomfield, Colorado, USA",USA,False,...,-15.5500,0.1250,-1.0000,neither,KO/TKO,1.0,3:01,181.0,0,Medium
4715,Clay Guida,Shannon Gugerty,-420.0,335.0,23.8095,335.0000,2010-03-21,"Broomfield, Colorado, USA",USA,False,...,-22.3500,0.6500,-2.1500,neither,SUB,2.0,3:40,520.0,0,Light
4716,Eliot Marshall,Vladimir Matyushenko,145.0,-165.0,145.0000,60.6061,2010-03-21,"Broomfield, Colorado, USA",USA,False,...,-4.8333,-0.5000,2.1667,neither,S-DEC,3.0,5:00,900.0,1,Heavy
4717,Brendan Schaub,Chase Gormley,-260.0,220.0,38.4615,220.0000,2010-03-21,"Broomfield, Colorado, USA",USA,False,...,-4.0000,1.0000,1.0000,neither,KO/TKO,1.0,0:47,47.0,0,Heavy


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Create Fighters Database

In [12]:
import pandas as pd

# Ensure 'Date' column is in datetime format
ufc_df['Date'] = pd.to_datetime(ufc_df['Date'])

# Identify columns specific to Blue and Red corners (excluding fighter names)
blue_columns = [col for col in ufc_df.columns if col.startswith('Blue') and col != 'BlueFighter']
red_columns  = [col for col in ufc_df.columns if col.startswith('Red') and col != 'RedFighter']

# Select shared columns and include corner-specific data
shared_columns = ['Date', 'Gender', 'WeightClassMap', 'WeightClass', 'Location', 'label']

# Create Blue corner dataframe with shared + Blue-specific columns
ufc_blue = ufc_df[shared_columns + ['BlueFighter'] + blue_columns].copy()
# Create Red corner dataframe with shared + Red-specific columns
ufc_red  = ufc_df[shared_columns + ['RedFighter'] + red_columns].copy()

# Standardize column names: rename fighter columns to 'Fighter' and remove corner prefixes
ufc_blue.columns = shared_columns + ['Fighter'] + [col.replace('Blue', '') for col in blue_columns]
ufc_red.columns  = shared_columns + ['Fighter'] + [col.replace('Red', '') for col in red_columns]

# Add 'Year' column extracted from date
ufc_blue['Year'] = ufc_blue['Date'].dt.year
ufc_red['Year']  = ufc_red['Date'].dt.year

# Add 'Corner' identifier to each dataset
ufc_blue['Corner'] = 'Blue'
ufc_red['Corner']  = 'Red'

# Combine both corners into a single DataFrame and sort: most recent fights first, Red before Blue
fighters_df = pd.concat([ufc_red, ufc_blue], ignore_index=True)
fighters_df = fighters_df.sort_values(by=['Date', 'Corner'], ascending=[False, True]).reset_index(drop=True)

# Add 'Win' column: mark as 'Yes' or 'No' based on corner and fight outcome
fighters_df['Win'] = ((fighters_df['Corner'] == 'Red') & (fighters_df['label'] == 0)) | \
                     ((fighters_df['Corner'] == 'Blue') & (fighters_df['label'] == 1))
fighters_df['Win'] = fighters_df['Win'].map({True: 'Yes', False: 'No'})

# Ensure fight stats are integers (fill missing values as 0)
fighters_df[['Wins', 'Losses', 'Draws']] = fighters_df[['Wins', 'Losses', 'Draws']].fillna(0).astype(int)

# Add 'Record' column: combine wins, losses, and draws as 'W-L-D'
fighters_df['Record'] = (
    fighters_df['Wins'].astype(str) + '-' +
    fighters_df['Losses'].astype(str) + '-' +
    fighters_df['Draws'].astype(str)
)

# Add 'TotalFights' column: total number of fights
fighters_df['TotalFights'] = fighters_df['Wins'] + fighters_df['Losses'] + fighters_df['Draws']

# Add 'WinRatio' column: proportion of wins over total fights (avoid div by zero)
fighters_df['WinRatio'] = fighters_df['Wins'] / fighters_df['TotalFights'].replace(0, 1)

# Add 'WinRatioPercentage' column: win ratio as percentage string
fighters_df['WinRatioPercentage'] = (fighters_df['WinRatio'] * 100).round(1).astype(str) + '%'

# Add 'FinishRate' column: proportion of wins by KO, submission, or doctor stoppage
fighters_df['FinishRate'] = (fighters_df['WinsByKO'] + fighters_df['WinsBySubmission'] + fighters_df['WinsByTKODoctorStoppage']) / fighters_df['Wins'].replace(0, 1)

# Add 'HeightReachRatio' column: height divided by reach
fighters_df['HeightReachRatio'] = fighters_df['HeightCms'] / fighters_df['ReachCms']

# Add 'UFCFights' column: total number of fights the fighter appears in dataset
fighters_df['UFCFights'] = fighters_df['Fighter'].map(fighters_df['Fighter'].value_counts())

# Add 'KOPerFight' column: KO + doctor stoppage wins per total fights
fighters_df['KOPerFight'] = (fighters_df['WinsByKO'] + fighters_df['WinsByTKODoctorStoppage']) / fighters_df['TotalFights'].replace(0, 1)

# Add 'SubPerFight' column: submission wins per total fights
fighters_df['SubPerFight'] = fighters_df['WinsBySubmission'] / fighters_df['TotalFights'].replace(0, 1)

# Drop unnecessary or redundant columns
fighters_df = fighters_df.drop(columns=['Odds', 'label', 'ExpectedValue', 'Corner'])

# Preview key columns (example slice to check final result)
fighters_df[['Date', 'Fighter', 'Gender', 'WeightClassMap', 'WeightClass', 'Year', 'Win', 'Record', 'WinRatio', 'UFCFights', 'TotalTitleBouts']]

,Date,Fighter,Gender,WeightClassMap,WeightClass,Year,Win,Record,WinRatio,UFCFights,TotalTitleBouts
0,2024-12-14,Joaquin Buckley,MALE,Medium,Welterweight,2024,Yes,10-4-0,0.714286,10,0
1,2024-12-14,Billy Quarantillo,MALE,Light,Featherweight,2024,No,7-4-0,0.636364,6,0
2,2024-12-14,Bruno Silva,MALE,Light,Flyweight,2024,No,4-2-0,0.666667,14,0
3,2024-12-14,Dustin Jacoby,MALE,Heavy,Light Heavyweight,2024,Yes,8-6-1,0.533333,10,0
4,2024-12-14,Daniel Marcos,MALE,Light,Bantamweight,2024,Yes,4-0-0,1.000000,3,0
...,...,...,...,...,...,...,...,...,...,...,...
9353,2010-03-21,Alessio Sakara,MALE,Medium,Middleweight,2010,Yes,5-5-0,0.500000,3,0
9354,2010-03-21,Clay Guida,MALE,Light,Lightweight,2010,Yes,5-5-0,0.500000,25,0
9355,2010-03-21,Eliot Marshall,MALE,Heavy,Light Heavyweight,2010,No,3-0-0,1.000000,3,0
9356,2010-03-21,Brendan Schaub,MALE,Heavy,Heavyweight,2010,Yes,0-1-0,0.000000,8,1


In [13]:
len(fighters_df['Fighter'].unique())

1856

In [14]:
# Filter out fighters with less than 4 UFC fights to exclude lesser-known fighters
fighters_df = fighters_df[fighters_df['UFCFights'] >= 4].copy()

In [15]:
len(fighters_df['Fighter'].unique())

912

In [16]:
# Filter last fight per year for each fighter

# Step 1: sort by date descending (this was done before, but we keep it for safety)
fighters_df = fighters_df.sort_values(by=['Fighter', 'Year', 'Date'], ascending=[True, True, False])

# Step 2: assign a counter per Fighter and Year
fighters_df['FightRankInYear'] = fighters_df.groupby(['Fighter', 'Year']).cumcount() + 1

# Step 3: filter only the last fight of the year (i.e., the first after sorting)
fighters_df = fighters_df[fighters_df['FightRankInYear'] == 1].copy()

# Step 4: drop the auxiliary column if not needed
fighters_df = fighters_df.drop(columns=['FightRankInYear'])

# Step 5 (optional): reset index for a clean DataFrame
fighters_df = fighters_df.reset_index(drop=True)

# Final result
fighters_df[['Date', 'Fighter', 'Year', 'Record', 'WinRatio']]

,Date,Fighter,Year,Record,WinRatio
0,2022-10-22,AJ Dobson,2022,1-1-0,0.500000
1,2023-08-12,AJ Dobson,2023,1-2-0,0.333333
2,2024-03-23,AJ Dobson,2024,2-2-0,0.500000
3,2022-08-20,AJ Fletcher,2022,1-1-0,0.500000
4,2023-09-23,AJ Fletcher,2023,2-2-0,0.500000
...,...,...,...,...,...
4736,2014-10-04,Zubaira Tukhugov,2014,1-0-0,1.000000
4737,2015-12-10,Zubaira Tukhugov,2015,2-0-0,1.000000
4738,2016-05-14,Zubaira Tukhugov,2016,3-0-0,1.000000
4739,2020-02-22,Zubaira Tukhugov,2020,3-1-1,0.600000


## Data Cleaning

In [17]:
# Null values check
nulls = fighters_df.isnull().sum()
print("\nNull values per column:\n", nulls[nulls > 0])

# Duplicate analysis
duplicates = fighters_df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

Null values per column:
 Series([], dtype: int64)

Duplicate rows: 0

In [18]:
# Drop rows with any remaining missing values
print(f"➡️ Before dropna: {fighters_df.shape}")
fighters_df.dropna(inplace=True)
print(f"✅ After dropna: {fighters_df.shape}")

➡️ Before dropna: (4741, 41)

✅ After dropna: (4741, 41)

In [19]:
# Save the cleaned file
fighters_df.to_csv(f'{project_root}/data/processed/fighters_df.csv', index=False)
logger.info("✅ Fighters file saved as 'fighters_df.csv'.")

INFO:src.helpers:✅ Fighters file saved as 'fighters_df.csv'.


<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

# Initialize UFCPredictor

In [20]:
# Define the path to the CSV file
file_path = os.path.join(project_root, 'data', 'processed', 'fighters_df.csv')

# Load the CSV into a DataFrame
try:
    fighters_df = pd.read_csv(file_path)
    logger.info("✅ Fighters Data loaded successfully.")
except Exception as e:
    logger.error(f"❌ Error loading data: {e}")

INFO:src.helpers:✅ Fighters Data loaded successfully.


In [21]:
ufc_predictor = UFCPredictor(fighters_df, ufc_data, ufc_data_no_odds, verbose=False)

In [22]:
ufc_predictor

🥋<UFCPredictor>🥋
  📊 Fighters loaded       : 912
  🏋️‍♂️ Weight classes       : 12
  🧠 Models with odds      : AdaBoost, Extra Trees, Gradient Boosting, K-Nearest Neighbors, Logistic Regression, Naive Bayes, Neural Network, Quadratic Discriminant Analysis, Random Forest, Support Vector Machine, XGBoost
  🚫 Models no odds        : AdaBoost (no_odds), Extra Trees (no_odds), Gradient Boosting (no_odds), K-Nearest Neighbors (no_odds), Logistic Regression (no_odds), Naive Bayes (no_odds), Neural Network (no_odds), Quadratic Discriminant Analysis (no_odds), Random Forest (no_odds), Support Vector Machine (no_odds), XGBoost (no_odds)
  ⭐ Default with odds     : nn_best
  ⭐ Default no odds       : xgb_best_no_odds
  🔢 Numerical features    : 25 (with odds), 24 (no odds)
3 (with odds), 3 (no odds)
  🛠️  Scaler               : StandardScaler

# Forecasting Experiments

<div style="text-align: center;">
  🔹 <img src="../img/ufc_logo.png" width="50" /> 🔹
</div>

## Experiment: Pereira 2024 (Red) vs Pereira 2023 (Blue): Odds model with 'OddsDif = 0' vs No Odds model

In [23]:
ufc_predictor.get_fighter_stats('Alex Pereira', 2023)

Date                                2023-11-11 00:00:00
Gender                                             MALE
WeightClassMap                                    Heavy
WeightClass                           Light Heavyweight
Location                   New York City, New York, USA
Fighter                                    Alex Pereira
CurrentLoseStreak                                     0
CurrentWinStreak                                      1
Draws                                                 0
AvgSigStrLanded                                    5.23
AvgSigStrPct                                       0.63
AvgSubAtt                                           0.3
AvgTDLanded                                        0.17
AvgTDPct                                            1.0
LongestWinStreak                                      4
Losses                                                1
TotalRoundsFought                                    16
TotalTitleBouts                                 

In [24]:
ufc_predictor.get_fighter_stats('Alex Pereira', 2024)

Date                             2024-10-05 00:00:00
Gender                                          MALE
WeightClassMap                                 Heavy
WeightClass                        Light Heavyweight
Location                   Salt Lake City, Utah, USA
Fighter                                 Alex Pereira
CurrentLoseStreak                                  0
CurrentWinStreak                                   4
Draws                                              0
AvgSigStrLanded                                 5.46
AvgSigStrPct                                    0.63
AvgSubAtt                                        0.3
AvgTDLanded                                     0.14
AvgTDPct                                         1.0
LongestWinStreak                                   4
Losses                                             1
TotalRoundsFought                                 21
TotalTitleBouts                                    5
WinsByDecisionMajority                        

In [25]:
result_odds = ufc_predictor.predict(
    ('Alex Pereira', 2024),
    ('Alex Pereira', 2023),
    1,
    'Neural Network',
    0,
    0,
)

result_no_odds = ufc_predictor.predict(
    ('Alex Pereira', 2024),
    ('Alex Pereira', 2023),
    1,
    'Logistic Regression (no_odds)',
)

In [26]:
print_prediction_result_notebooks(result_odds)


🏆 UFC FIGHT PREDICTION RESULT
----------------------------------------------------------------------
🔴 RED CORNER (Favorite): Alex Pereira (2024)
----------------------------------------------------------------------
Record               : 8-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Odds                 : 0
Total Fights        : 9 | Title Bouts: 5
Age                : 37
Height             : 193.04 cm | Reach: 200.66 cm
Height/Reach Ratio : 0.962
Win Ratio          : 0.889 | Finish Rate: 0.750
KO per Fight       : 0.667 | Sub per Fight: 0.000
Avg Sig Str Landed : 5.460
Avg Sub Att        : 0.300 | Avg TD Landed: 0.140

🔵 BLUE CORNER (Underdog): Alex Pereira (2023)
----------------------------------------------------------------------
Record               : 5-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Odds                 : 0
Total Fights        : 6 | Title Bouts: 4
Age                : 36
Height             : 193.04 cm | Reach: 200.66 cm
Height

In [27]:
print_prediction_result_notebooks(result_no_odds)


🏆 UFC FIGHT PREDICTION RESULT
----------------------------------------------------------------------
🔴 RED CORNER (Favorite): Alex Pereira (2024)
----------------------------------------------------------------------
Record               : 8-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Total Fights        : 9 | Title Bouts: 5
Age                : 37
Height             : 193.04 cm | Reach: 200.66 cm
Height/Reach Ratio : 0.962
Win Ratio          : 0.889 | Finish Rate: 0.750
KO per Fight       : 0.667 | Sub per Fight: 0.000
Avg Sig Str Landed : 5.460
Avg Sub Att        : 0.300 | Avg TD Landed: 0.140

🔵 BLUE CORNER (Underdog): Alex Pereira (2023)
----------------------------------------------------------------------
Record               : 5-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Total Fights        : 6 | Title Bouts: 4
Age                : 36
Height             : 193.04 cm | Reach: 200.66 cm
Height/Reach Ratio : 0.962
Win Ratio          : 0.833 | 

## Experiment: Pereira 2023 (Red) vs Pereira 2024 (Blue): Odds model with 'OddsDif = 0' vs No Odds model

In [28]:
result_odds = ufc_predictor.predict(
    ('Alex Pereira', 2023),
    ('Alex Pereira', 2024),
    1,
    'Neural Network',
    0,
    0,
)

result_no_odds = ufc_predictor.predict(
    ('Alex Pereira', 2023),
    ('Alex Pereira', 2024),
    1,
    'Logistic Regression (no_odds)',
)

In [29]:
print_prediction_result_notebooks(result_odds)


🏆 UFC FIGHT PREDICTION RESULT
----------------------------------------------------------------------
🔴 RED CORNER (Favorite): Alex Pereira (2023)
----------------------------------------------------------------------
Record               : 5-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Odds                 : 0
Total Fights        : 6 | Title Bouts: 4
Age                : 36
Height             : 193.04 cm | Reach: 200.66 cm
Height/Reach Ratio : 0.962
Win Ratio          : 0.833 | Finish Rate: 0.600
KO per Fight       : 0.500 | Sub per Fight: 0.000
Avg Sig Str Landed : 5.230
Avg Sub Att        : 0.300 | Avg TD Landed: 0.170

🔵 BLUE CORNER (Underdog): Alex Pereira (2024)
----------------------------------------------------------------------
Record               : 8-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Odds                 : 0
Total Fights        : 9 | Title Bouts: 5
Age                : 37
Height             : 193.04 cm | Reach: 200.66 cm
Height

In [30]:
print_prediction_result_notebooks(result_no_odds)


🏆 UFC FIGHT PREDICTION RESULT
----------------------------------------------------------------------
🔴 RED CORNER (Favorite): Alex Pereira (2023)
----------------------------------------------------------------------
Record               : 5-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Total Fights        : 6 | Title Bouts: 4
Age                : 36
Height             : 193.04 cm | Reach: 200.66 cm
Height/Reach Ratio : 0.962
Win Ratio          : 0.833 | Finish Rate: 0.600
KO per Fight       : 0.500 | Sub per Fight: 0.000
Avg Sig Str Landed : 5.230
Avg Sub Att        : 0.300 | Avg TD Landed: 0.170

🔵 BLUE CORNER (Underdog): Alex Pereira (2024)
----------------------------------------------------------------------
Record               : 8-1-0
Weight Class        : Light Heavyweight | Stance: Orthodox
Total Fights        : 9 | Title Bouts: 5
Age                : 37
Height             : 193.04 cm | Reach: 200.66 cm
Height/Reach Ratio : 0.962
Win Ratio          : 0.889 | 

## 🧪 Experiment: Pereira 2024 vs Pereira 2023 — Odds Model vs No-Odds Model

### Experiment summary

We compared model predictions across two cross-match scenarios:

- **Pereira 2024 (Red) vs Pereira 2023 (Blue)**
- **Pereira 2023 (Red) vs Pereira 2024 (Blue)**

For each matchup, we tested:

- **Odds model** (`OddsDif = 0` → simulates a market-even fight; Neural Network, good accuracy, high F1 score)  
- **No-odds model** (Logistic Regression, good accuracy, high F1 score)

We recorded the predicted win probabilities for the Red corner.

### Key results

| Matchup                            | Odds Model (Red Prob) | No-Odds Model (Red Prob) |
|-------------------------------------|------------------------|---------------------------|
| 2024 (Red) vs 2023 (Blue)          | 50.7%                 | 69.3%                    |
| 2023 (Red) vs 2024 (Blue)          | 46.0%                 | 61.0%                    |

### Brief conclusions

✅ Changing corners (Red ↔ Blue) impacts predictions, even when the fighters are identical.  
✅ The no-odds model tends to amplify differences, producing more extreme probability swings.  
✅ Setting `OddsDif = 0` **is not the same as removing odds entirely** — it simulates a market draw, but the odds-trained model still incorporates historical patterns.  
✅ Small shifts in features (like WinStreakDif, AgeDif) can cause ~5–7% changes in predicted probabilities.  
✅ The predictor realistically reflects a Red corner bias, historically correlated with favorites.

<div style="text-align: center;">
     <img src="../img/ufc_logo.png" width="800" /> 
</div>